## LSTM Model

In [4]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from termcolor import colored

Using TensorFlow backend.


In [6]:
# Load cleaned data
train_data = pd.read_csv('clean_train.csv')
test_data = pd.read_csv('clean_test.csv')

### Tokenization

In [7]:
print(colored("Tokenizing and padding data", "yellow"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(train_data['Clean_tweet'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(train_data['Clean_tweet'].astype(str).values)

max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(test_data['Clean_tweet'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


### Build LSTM Model

In [8]:
print(colored("Creating the LSTM model", "yellow"))
model = Sequential()
model.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout = 0.2))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Creating the LSTM model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 40, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 650,754
Trainable params: 650,754
Non-trainable params: 0
_________________________________________________________________


### Train the Model

In [9]:
print(colored("Training the LSTM model", "green"))
history = model.fit(train_tweets, 
                    pd.get_dummies(train_data['Sentiment']).values, 
                    epochs = 10, 
                    batch_size = 128, 
                    validation_split = 0.2)
print(colored(history, "green"))

Training the LSTM model


//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1024000 samples, validate on 256000 samples
Epoch 1/10
1024000/1024000 [==============================] - 4460s 4ms/step - loss: 0.4801 - accuracy: 0.7677 - val_loss: 0.4625 - val_accuracy: 0.7785
Epoch 2/10
1024000/1024000 [==============================] - 3674s 4ms/step - loss: 0.4617 - accuracy: 0.7791 - val_loss: 0.4576 - val_accuracy: 0.7811
Epoch 3/10
1024000/1024000 [==============================] - 3063s 3ms/step - loss: 0.4553 - accuracy: 0.7829 - val_loss: 0.4536 - val_accuracy: 0.7835
Epoch 4/10
1024000/1024000 [==============================] - 3251s 3ms/step - loss: 0.4507 - accuracy: 0.7857 - val_loss: 0.4522 - val_accuracy: 0.7843
Epoch 5/10
1024000/1024000 [==============================] - 3224s 3ms/step - loss: 0.4466 - accuracy: 0.7880 - val_loss: 0.4493 - val_accuracy: 0.7861
Epoch 6/10
1024000/1024000 [==============================] - 3201s 3ms/step - loss: 0.4428 - accuracy: 0.7901 - val_loss: 0.4480 - val_accuracy: 0.7872
Epoch 7/10
1024000/1024000 [=

### Test the Model


In [10]:
print(colored("Testing the LSTM model", "green"))
score, accuracy = model.evaluate(test_tweets, 
                                 pd.get_dummies(test_data['Sentiment']).values, 
                                 batch_size = 128)
print("Test accuracy: {}".format(accuracy))

Testing the LSTM model
320000/320000 [==============================] - 270s 845us/step
Test accuracy: 0.7872593998908997
